### 任务一
在任务一中，本小组选择Resnet18图像分类网络，实现了将Pytorch模型分别转换成Caffe模型与ONNX模型，并针对同一张测试图片，测试打印了三种模型下的推理结果置信度以及对于同一测试图片进行图像推理的模型吞吐量。置信度结果相差不大，可以忽略Pytorch向Caffe模型、ONNX模型转换后的精度损失。具体实现流程如下：

### 1.模型载入与转换
#### 1.1 导入头文件

In [1]:
#导入pytorch相关头文件
import torch
import torchvision.models as models

#导入caffe相关头文件
import sys
sys.path.append('/root/brocolli')
from brocolli.converter.pytorch_caffe_parser import PytorchCaffeParser
import caffe
import numpy as np

#导入onnx相关头文件
import onnx
import onnxruntime as ort
import numpy as np
import cv2


#### 1.2 导入Resnet18神经网络的pytorch模型

In [2]:
net = models.resnet18(pretrained=True)
net.eval()

/root/miniconda3/envs/caffe_0618/lib/python3.7/site-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and may be removed in the future, "
/root/miniconda3/envs/caffe_0618/lib/python3.7/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

#### 1.3 实现pytorch模型转换成caffe模型

In [3]:
x = torch.rand(1, 3, 224, 224)
pytorch_parser = PytorchCaffeParser(net, x)
pytorch_parser.convert()
pytorch_parser.save('resnet18')

2023-06-20 17:25:54.244 | DEBUG    | brocolli.converter.pytorch_graph:print_tabular:119 - 
opcode         name                   target                                                      args                                   kwargs
-------------  ---------------------  ----------------------------------------------------------  -------------------------------------  --------
placeholder    x                      x                                                           ()                                     {}
call_module    conv1                  conv1                                                       (x,)                                   {}
call_module    bn1                    bn1                                                         (conv1,)                               {}
call_module    relu                   relu                                                        (bn1,)                                 {}
call_module    maxpool                maxpool              

#### 1.4 实现pytorch模型转换成onnx模型

In [4]:
net = models.resnet18(pretrained=True)
x = torch.rand(1, 3, 224, 224)
torch.onnx.export(net, x, 'resnet18.onnx')

## 2. 准确性损失评估
### 2.1 载入图片并进行预处理

In [5]:
from PIL import Image as PILImage
from timm.data import resolve_data_config
from timm.data.transforms_factory import create_transform
from IPython.display import Image as IPythonImage

config = resolve_data_config({}, model=net)
transform = create_transform(**config)
#载入图片
filename =  "../images/dog.jpg"
IPythonImage(filename=filename)

FileNotFoundError: [Errno 2] No such file or directory: '../images/dog.jpg'

In [ ]:
#图像预处理
print(transform)
img = PILImage.open(filename).convert('RGB')
tensor = transform(img).unsqueeze(0) # transform and add batch dimension

#载入标签文件
with open("../data/imagenet_classes.txt", "r") as f:
    categories = [s.strip() for s in f.readlines()]

Compose(
    Resize(size=256, interpolation=bicubic, max_size=None, antialias=None)
    CenterCrop(size=(224, 224))
    ToTensor()
    Normalize(mean=tensor([0.4850, 0.4560, 0.4060]), std=tensor([0.2290, 0.2240, 0.2250]))
)


### 2.2 图像推理
将预处理后的图像输入到推理框架之中进行图像推理，分别测试pytorch、caffe、onnx三个框架模型中的推理结果，比较pytorch分别向caffe模型、onnx模型转换后的精度损失
#### 2.2.1 Pytorch（Top-5）

In [ ]:
import time

net = models.resnet18(pretrained=True)
net.eval()
with torch.no_grad():
    out = net(tensor)

NUM_IMAGES = 1000  # 测试推理的图像数量
start_time = time.time()
for _ in range(NUM_IMAGES):
    probabilities = torch.nn.functional.softmax(out[0], dim=0)
end_time = time.time()

#计算打印置信度top5和对应的标签
probabilities = torch.nn.functional.softmax(out[0], dim=0)
top5_prob, top5_catid = torch.topk(probabilities, 5)
for i in range(top5_prob.size(0)):
    print(categories[top5_catid[i]], top5_prob[i].item())

#计算打印吞吐量
total_time = end_time - start_time
throughput = NUM_IMAGES / total_time
print(f'Throughput: {throughput} images/second')

NameError: name 'models' is not defined

#### 2.2.2 Caffe（Top-5）

In [ ]:
# 定义模型文件和权重文件的路径
model_file = './resnet18.prototxt'
weight_file = './resnet18.caffemodel'
# 加载模型和权重
net = caffe.Net(model_file, weight_file, caffe.TEST)
# 定义输入数据的形状
net.blobs['x'].reshape(1, 3, 224, 224)
# 将输入数据设置为网络的输入
net.blobs['x'].data[...] = tensor
net.forward()# 进行前向推理
output = net.blobs['fc'].data[0] #获取网络的输出

NUM_IMAGES = 1000  # 测试推理的图像数量
start_time = time.time()
for _ in range(NUM_IMAGES):
    softmax = np.exp(output) / np.sum(np.exp(output))
    sorted_indices = np.argsort(-softmax)
end_time = time.time()

#计算打印置信度top5和对应的标签
top_5_indices = sorted_indices[:5]
for idx in top_5_indices:
    print(categories[idx], softmax[idx])

#计算打印吞吐量
total_time = end_time - start_time
throughput = NUM_IMAGES / total_time
print(f'Throughput: {throughput} images/second')

W20230620 01:05:20.321411   537 _caffe.cpp:122] DEPRECATION WARNING - deprecated use of Python interface
W20230620 01:05:20.321446   537 _caffe.cpp:123] Use this instead (with the named "weights" parameter):
W20230620 01:05:20.321449   537 _caffe.cpp:125] Net('./resnet18.prototxt', 1, weights='./resnet18.caffemodel')


Samoyed 0.89802784
white wolf 0.0439865
Arctic fox 0.03794145
Pomeranian 0.0047865086
West Highland white terrier 0.0033660661
Throughput: 16.401650354125227 images/second


#### 2.2.3 ONNX（Top-5）

In [ ]:
model = ort.InferenceSession("resnet18.onnx")
input_name = model.get_inputs()[0].name
output_name = model.get_outputs()[0].name
outputs = model.run([output_name], {input_name: tensor.numpy()})
output_tensor = torch.Tensor(outputs[0])

NUM_IMAGES = 1000  # 测试推理的图像数量
start_time = time.time()
for _ in range(NUM_IMAGES):
    probabilities = torch.nn.functional.softmax(output_tensor[0], dim=0)
end_time = time.time()

#计算打印置信度top5和对应的标签
probabilities = torch.nn.functional.softmax(output_tensor[0], dim=0)
top5_prob, top5_catid = torch.topk(probabilities, 5)
for i in range(top5_prob.size(0)):
    print(categories[top5_catid[i]], top5_prob[i].item())

#计算打印吞吐量
total_time = end_time - start_time
throughput = NUM_IMAGES / total_time
print(f'Throughput: {throughput} images/second')

Samoyed 0.8980278968811035
white wolf 0.04398662969470024
Arctic fox 0.03794185072183609
Pomeranian 0.004786513280123472
West Highland white terrier 0.0033660889603197575
Throughput: 93.2584663389353 images/second


> 综上可知，Pytorch2Caffe、Pytorch2onnx的模型转换成功，精度损失可以忽略